In [1]:
import requests
import urllib
import pickle
import os
import json

In [2]:
from multiprocessing import Pool

In [3]:
class MSFaceAPIClient():
    def __init__(self, subscription_key, location):
        self._subscription_key = subscription_key
        self._location = location
        
    def face_detect_local(self, image_path):
        headers = {
            # Request headers
            'Content-Type': 'application/octet-stream',
            'Ocp-Apim-Subscription-Key': self._subscription_key,
        }

        params = urllib.parse.urlencode({
            # Request parameters
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'true',
            'returnFaceAttributes': "age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise"
        })

        try:
            url = "https://" + self._location + ".api.cognitive.microsoft.com/face/v1.0/detect"
            return requests.post(url, params=params, headers=headers, data=open(image_path, 'rb')).json()
        except Exception as e:
            print("Detect Face Error: " + str(e))
            return None

In [ ]:
face_api_client = MSFaceAPIClient(
    subscription_key = "1f5456936f2e4369b2226440b624dd00",
    location = "westcentralus"
)

with open('selfie_shortcodes_10055.pkl', 'rb') as f:
    shortcodes = pickle.load(f)

in_path = "/mnt/e/Data/selfies/images"
out_path = "/mnt/e/Data/selfies/face_api_json"

for shortcode in shortcodes:
    if not os.path.exists(os.path.join(out_path, shortcode + ".json")):
        image_path = os.path.join(in_path, shortcode + ".jpg")
        data = face_api_client.face_detect_local(image_path)
        if data is not None:
            if len(data) > 0:
                with open(os.path.join(out_path, shortcode + ".json"), "w") as fout:
                    json.dump(data, fout)
    

Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrUNMHV63.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrTVDlQhr.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrTDGhpLC.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTqrGgBtgG.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTpfkHFPdW.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BtiyFRlAQvi.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrTwzley9.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrTpClVot.jpg'
Detect Face Error: [Errno 2] No such file or directory: '/mnt/e/Data/selfies/images/BuTrTc0guZ2.jpg'
